In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
#Box(0,255,shape=(3,3), dtype=int).sample()

In [42]:
class ChargeEnv(Env):
    def __init__(self):
        # Actions we can take, down, stay, up
        self.action_space = Discrete(3)
        # Observation box
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        # Set start SOC
        self.state = 20 + random.randint(-3,3)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 1.18,
                                               1.11, 1.42, 5.87, 0.71, 1.44]))
        # Set current index
        self.currentIndex = 0
        self.currentprice = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        self.profit = 0
        # Set median
        self.mid = np.median(self.priceArray)
        self.open = 0
        self.rate = 1
        # Set total time 
        self.time = 23 
        
    def step(self, action):
        # Apply action
        # 0 -1 = -1 sell
        # 1 -1 = 0 hold
        # 2 -1 = 1 buy 
        
        
        # Update the state
        self.state += (action - 1) * self.power 
       
        self.currentprice = self.priceArray[self.currentIndex]
        # Reduce time by 1 h
        self.time -= 1 
        
        # If state exceeds boundaries after the action due to power limit, set state to 100 or 0
        if 100 < self.state < (self.max_state + self.power):
            self.rate = (self.state[0] - self.max_state) / self.power
            self.state = 100
            #action = 0
        elif (0 - self.power) < self.state < 0:
            self.rate = (0 - self.state[0]) / self.power 
            self.state = 0
            #action = 2
        else: 
            self.rate = 1
         
        self.change = self.state / 100
        
        # Lose money when buy, and get when sell
        self.temp = -1 * self.currentprice * (action - 1) * self.rate
        
        # Calculate cumulative profit
        self.profit += self.temp    
                 
            
        if self.time > 16:
            if self.profit >= 0:
                if self.state >= 80:
                    reward = 30 / self.time
                elif self.state >= 60:
                    reward = 28 / self.time
                elif self.state >= 40:
                    reward = 20 / self.time
                elif self.state >= 20:
                    reward = 16 / self.time
                else:
                    reward = 10 / self.time
                
            else:
                if self.state >= 80:
                    reward = 20 / self.time
                elif self.state >= 60:
                    reward = 16 / self.time
                elif self.state >= 40:
                    reward = -20 / self.time
                elif self.state >= 20:
                    reward = -28 / self.time
                else:
                    reward = -30 / self.time
       
        if self.time > 9:
            if self.profit >= 0:
                if self.state >= 80:
                    reward = 18 / self.time
                elif self.state >= 60:
                    reward = 12 / self.time
                else:
                    reward = -5 / self.time
            else:
                if self.state >= 80:
                    reward = 1 / self.time
                elif self.state >= 60:
                    reward = -10 / self.time
                else:
                    reward = -18 / self.time
                    
        if self.time > 4:
            if self.state >= 80:
                if self.profit >= 0:
                    reward = 20 / self.time
                else:
                    reward = -5 / self.time
            elif self.state >= 60:
                if self.profit >= 0:
                    reward = 10 / self.time
                else:
                    reward = -10 / self.time
            else:
                if self.profit >= 0:
                    reward = -15 / self.time
                else:
                    reward = -20 / self.time
        else:
            if self.state >= 80:
                reward = 30 
            else:
                reward = -91 
    
        
            
        
        # Give reward based on the profit

            

        
        # Move to the next price
        self.currentIndex += 1
        
        # Set Low boundary and Up boundary
        #Set end state
        
        if self.state <= -20 or self.state >= 120:
            #done = True
            reward = -50 * (23 - self.time)
        if self.time <= 0:
            done = True
            if self.state < 80:
                if self.profit < 0: 
                    reward += -300
                else:
                    reward += -300
            else:
                if self.profit < 0:
                    reward += -100
                else:
                    reward += 500

        elif self.profit >= 0 and self.state >= 90:
            done = True
            reward = 1000
        else:#1
            done = False       

        

        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info#, self.profit

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset shower temperature
        self.state = np.array([20 + random.randint(-3,3)])#.astype(float)
        # Set max SOC
        self.max_state = 100
        # Set power
        self.power = 20
        # Set price data
        self.priceArray = np.float32(np.array([36.22, 27.98, 4.6, 9.38, -0.43, 2.26, 4.02, 8.48, 18.73,
                                               20.64, 22.99, 18.93, 16.67, 9.51, 3.59, 2.31, 2.78, 6.3, 1.18,
                                               1.11, 1.42, 5.87, 0.71, 1.44]))
        # Set current index
        self.currentIndex = 0
        self.currentprice = 0
        # Set temp 
        self.temp = 0
        self.change = 0
        self.rate = 1
        self.profit = 0
        # Set median
        self.mid = np.median(self.priceArray)
        self.open = 0
        # Set total time 
        self.time = 23 
        return self.state

In [43]:
env=ChargeEnv()

In [44]:
env.observation_space.sample()

array([4.488243], dtype=float32)

In [45]:
env.reset()

array([20])

In [46]:
#from stable_baselines3.common.env_checker import check_env

In [47]:
#check_env(env, warn=True)

In [50]:
episodes = 50
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))
env.close()

Episode:1 Score:-13951.396103896104 State:-20
Episode:2 Score:-2371.231829939067 State:[120]
Episode:3 Score:-921.8371330252136 State:20
Episode:4 Score:-11104.388895310916 State:140
Episode:5 Score:-779.1121987532591 State:0
Episode:6 Score:-7417.8979130389735 State:[-20]
Episode:7 Score:-13602.935577580314 State:-80
Episode:8 Score:-6796.990967365968 State:[-20]
Episode:9 Score:-3863.5340878376874 State:40
Episode:10 Score:932.1029055868452 State:100
Episode:11 Score:-6104.345029239766 State:0
Episode:12 Score:-1264.8066431977036 State:80
Episode:13 Score:-8308.992125199362 State:[20]
Episode:14 Score:-782.8724041266611 State:40
Episode:15 Score:-579.8227250690486 State:80
Episode:16 Score:-14050.681818181818 State:-100
Episode:17 Score:-651.2441288072093 State:20
Episode:18 Score:-4320.630430521491 State:40
Episode:19 Score:-1047.6292349291575 State:80
Episode:20 Score:-11186.651263854825 State:[-20]
Episode:21 Score:-10264.681609326346 State:-80
Episode:22 Score:-839.0840805221611 

In [51]:
log_path = os.path.join('Training', 'Logs')

In [52]:
model = PPO("MlpPolicy", env, learning_rate=0.001, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [53]:
model.learn(total_timesteps=400000)

Logging to Training\Logs\PPO_128
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 22.6     |
|    ep_rew_mean     | -5.5e+03 |
| time/              |          |
|    fps             | 291      |
|    iterations      | 1        |
|    time_elapsed    | 7        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 22.9        |
|    ep_rew_mean          | -4.02e+03   |
| time/                   |             |
|    fps                  | 316         |
|    iterations           | 2           |
|    time_elapsed         | 12          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.010351349 |
|    clip_fraction        | 0.159       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000333   

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 23           |
|    ep_rew_mean          | -737         |
| time/                   |              |
|    fps                  | 303          |
|    iterations           | 11           |
|    time_elapsed         | 74           |
|    total_timesteps      | 22528        |
| train/                  |              |
|    approx_kl            | 0.0052318107 |
|    clip_fraction        | 0.105        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.738       |
|    explained_variance   | -1.79e-06    |
|    learning_rate        | 0.001        |
|    loss                 | 6.87e+04     |
|    n_updates            | 100          |
|    policy_gradient_loss | -0.0117      |
|    value_loss           | 1.08e+05     |
------------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 23           |
|    ep_rew_mean          | -2.42e+03    |
| time/                   |              |
|    fps                  | 299          |
|    iterations           | 21           |
|    time_elapsed         | 143          |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0064387172 |
|    clip_fraction        | 0.132        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.515       |
|    explained_variance   | 0            |
|    learning_rate        | 0.001        |
|    loss                 | 2.07e+06     |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.0172      |
|    value_loss           | 5.52e+06     |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 23            |
|    ep_rew_mean          | -781          |
| time/                   |               |
|    fps                  | 306           |
|    iterations           | 31            |
|    time_elapsed         | 206           |
|    total_timesteps      | 63488         |
| train/                  |               |
|    approx_kl            | 0.00088459486 |
|    clip_fraction        | 0.00996       |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.114        |
|    explained_variance   | 0             |
|    learning_rate        | 0.001         |
|    loss                 | 2.57e+03      |
|    n_updates            | 300           |
|    policy_gradient_loss | -0.00111      |
|    value_loss           | 7.04e+03      |
-------------------------------------------
------------------------------------------
| rollout/                |      

-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 23            |
|    ep_rew_mean          | -779          |
| time/                   |               |
|    fps                  | 313           |
|    iterations           | 40            |
|    time_elapsed         | 261           |
|    total_timesteps      | 81920         |
| train/                  |               |
|    approx_kl            | 1.8694001e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.0215       |
|    explained_variance   | -1.19e-07     |
|    learning_rate        | 0.001         |
|    loss                 | 3e+03         |
|    n_updates            | 390           |
|    policy_gradient_loss | -3.05e-05     |
|    value_loss           | 6.48e+03      |
-------------------------------------------
------------------------------------------
| rollout/                |      

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 23        |
|    ep_rew_mean          | -779      |
| time/                   |           |
|    fps                  | 318       |
|    iterations           | 50        |
|    time_elapsed         | 321       |
|    total_timesteps      | 102400    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.00383  |
|    explained_variance   | -1.19e-07 |
|    learning_rate        | 0.001     |
|    loss                 | 3.41e+03  |
|    n_updates            | 490       |
|    policy_gradient_loss | 2.1e-09   |
|    value_loss           | 6.48e+03  |
---------------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 23            |
|    ep_rew_mean          | 

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 23        |
|    ep_rew_mean          | -779      |
| time/                   |           |
|    fps                  | 321       |
|    iterations           | 60        |
|    time_elapsed         | 381       |
|    total_timesteps      | 122880    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000671 |
|    explained_variance   | 0         |
|    learning_rate        | 0.001     |
|    loss                 | 1.65e+03  |
|    n_updates            | 590       |
|    policy_gradient_loss | 1.61e-09  |
|    value_loss           | 6.48e+03  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 23        |
|    ep_rew_mean          | -779      |


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 23        |
|    ep_rew_mean          | -779      |
| time/                   |           |
|    fps                  | 324       |
|    iterations           | 70        |
|    time_elapsed         | 442       |
|    total_timesteps      | 143360    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000388 |
|    explained_variance   | 0         |
|    learning_rate        | 0.001     |
|    loss                 | 3.62e+03  |
|    n_updates            | 690       |
|    policy_gradient_loss | 4.83e-10  |
|    value_loss           | 6.48e+03  |
---------------------------------------
---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 23        |
|    ep_rew_mean          | -779      |


KeyboardInterrupt: 

In [402]:
#evaluate_policy(model, env, n_eval_episodes=10, render=True)

In [403]:
#model.save('PPO')

In [54]:
episodes = 100
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _states = model.predict(obs)
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{} State:{}'.format(episode, score, n_state))
env.close()

Episode:1 Score:-779.1121987532591 State:[18]
Episode:2 Score:-779.1121987532591 State:[23]
Episode:3 Score:-779.1121987532591 State:[21]
Episode:4 Score:-779.1121987532591 State:[21]
Episode:5 Score:-779.1121987532591 State:[22]
Episode:6 Score:-779.1121987532591 State:[21]
Episode:7 Score:-779.1121987532591 State:[21]
Episode:8 Score:-779.1121987532591 State:[21]
Episode:9 Score:-779.1121987532591 State:[21]
Episode:10 Score:-779.1121987532591 State:[19]
Episode:11 Score:-779.1121987532591 State:[22]
Episode:12 Score:-779.1121987532591 State:[23]
Episode:13 Score:-779.1121987532591 State:[21]
Episode:14 Score:-779.1121987532591 State:[18]
Episode:15 Score:-779.1121987532591 State:[19]
Episode:16 Score:-779.1121987532591 State:[19]
Episode:17 Score:-779.1121987532591 State:[17]
Episode:18 Score:-779.1121987532591 State:[20]
Episode:19 Score:-779.1121987532591 State:[22]
Episode:20 Score:-779.1121987532591 State:[19]
Episode:21 Score:-779.1121987532591 State:[18]
Episode:22 Score:-779.

In [ ]:
env.close()

In [ ]:
#Save the model
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_model')

In [ ]:
model.save(PPO_path)

In [ ]:
del model

In [ ]:
model = PPO.load('PPO_model', env=env)

In [ ]:
#View logs in Tensorboard
training_log_path = os.path.join(log_path, 'PPO')
!tensorboard --logdir={training_log_path}